<a href="https://colab.research.google.com/github/victorprieto2265/cornerstone/blob/main/Cornerstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TODO



TODO

*  import computer modern typeface for use (consider open sans or interstate as an alternative)


# Import modules / libraries

In [ ]:
!pip install fpdf2

import sys
import pandas as pd
import warnings
warnings.simplefilter('default', DeprecationWarning)
from fpdf import FPDF
import fpdf
from base64 import b64encode
import IPython
from IPython.display import display, HTML

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Cornerstone Input

This section imports all of the input files, except tournament format,
which is handled by the tournament_format script.

TODO uncomment sys.exit at end of this script for testing

TODO figure out how to handle blank spaces and/or none in input file...
There was an unusual error where integers were being converted to floats
if one of the cells in a range was empty instead of string 'none'.

TODO apparently bracket names (and presumably other things) are case-sensitive.

TODO currently a global variable for checking for errors, which is a no no?


In [ ]:
error = 0

# checks for duplicates and (optional) character limits
def error_check(alist, listname, max_length=float('inf'), max_duplicates=1):
    global error  # TODO this is apparently a no no, but works for now
    print('\n')
    # duplicate checker
    seen = {}
    duplicates = []
    for item in alist:
        if item not in seen:
            seen[item] = 1
        else:
            seen[item] += 1
            if seen[item] > max_duplicates:
                duplicates.append(' - ' + item)
    if not duplicates:
        if max_duplicates > 1:
            print(f'No excess duplicates detected for {listname}.')
        else:
            print(f'No duplicates detected for {listname}.')
    else:
        # error = True
        error += 1
        print(f'\nDuplicates detected for {listname}:')
        print(*duplicates, sep='\n')

    # character length checker
    long_items = [' - ' + item for item in alist if len(item) > max_length]
    if not long_items:
        print(f'No character lengths exceeded for {listname}.')
    else:
        error += 1
        # error = True
        print(f'Character lengths exceeded for {listname}:')
        print(*long_items, sep='\n')

Function definition: analyze sheet and generate list from pandas dataframe.

In [ ]:
def analyze_input(sheet_name, df_dict):
    df = df_dict[sheet_name]
    temp_list = df.values.tolist()  # note: temp_list may be multiple lists!
    return temp_list

Prompt user for which tournament phase is being implemented. TODO consider in future to automatically evaluate tournament phase based on file name.

In [ ]:
# step 0 = identify which excel files to pull? (not implemented while testing)
print('\n***\nAre you uploading excel sheets for prelim schedule creation'
      + ' or rebracketing for playoffs?')
# tournament_phase = input('   enter "prelims" or "playoffs" => ')
tournament_phase = 'prelims'

while tournament_phase not in ['prelims', 'playoffs', 'super']:
    print('\n***incorrect input provided, please retry***')
    print('\nSelect either prelim schedule creation or '
          + 'rebracketing for playoffs:')
    tournament_phase = input('   enter "prelims" or "playoffs" => ')


***
Are you uploading excel sheets for prelim schedule creation or rebracketing for playoffs?


---

Upload the prelims_data excel spreadsheet from local computer, and create dictionary containing the sheet name and each dataframe.

TODO For now, this is commented out. While testing Cornerstone as I'm porting over to Colab, I will be directly pulling the excel file from a dedicated location in github.

In [ ]:
# from google.colab import files
# file_upload = files.upload()
# file_upload = list(file_upload.keys())[0]

# sheet_names = ['tournament format', 'list of teams', 'team codes',
#                 'group names', 'room assignments',
#                 'QR codes', 'text input']
# df_dict_prelim = pd.read_excel(file_upload,
#                                sheet_name=sheet_names)

In [ ]:
sheet_names = ['tournament format', 'list of teams', 'team codes',
                'group names', 'room assignments',
                'QR codes', 'text input']
url = "https://github.com/victorprieto2265/cornerstone/blob/main/data%20input/prelims_data.xlsx?raw=true"
df_dict_prelim = pd.read_excel(url,
                               sheet_name=sheet_names)


Using the analyze_input function, create a list from each tab of the spreadsheet corresponding to the relevant information (e.g. a list of room assignments). If in playoffs or superplayoffs, also create the relevant lists there.

In [ ]:
list_of_teams = analyze_input('list of teams', df_dict_prelim)
team_codes = analyze_input('team codes', df_dict_prelim)
room_assignments = analyze_input('room assignments', df_dict_prelim)
qr_codelist = analyze_input('QR codes', df_dict_prelim)
textlist = analyze_input('text input', df_dict_prelim)
tournament_format = analyze_input('tournament format', df_dict_prelim)

group_names = analyze_input('group names', df_dict_prelim)
prelim_group_names = [row[0] for row in group_names]

if tournament_phase in ['playoffs', 'super']:
    playoff_bracket_names = [row[1] for row in group_names]
    playoff_bracket_names = [x for x in playoff_bracket_names
                             if str(x) != 'nan']
if tournament_phase == 'super':
    super_bracket_names = [row[2] for row in group_names]
    super_bracket_names = [x for x in super_bracket_names
                             if str(x) != 'nan']

A dictionary containing general information about the tournament is also generated from the tournament_format sheet.

It's necessary to convert the datetime value from excel to a more suitable string before printing.

In [ ]:
format_dict = {}
for row in tournament_format:
    key = row[0]  # item, e.g. "tournament name"
    value = row[1]  # input, e.g. "2022 NSC"
    format_dict[key] = value

# convert Excel datetime format to more suitable date string
date = format_dict['tournament date'].strftime('%B %d, %Y')
format_dict['tournament date'] = date

Next is the analysis section for the text and QR code inputs. Did the user turn these on? If not, these sections are skipped.

TODO qr_codes used to have a NoEscape function, but that was because of pylatex and has been removed.

In [ ]:
if format_dict['QR codes'] == 'Y':
    qr_toggle = True

    qr_names = [code[0] for code in qr_codelist]
    qr_codes = [code[2] for code in qr_codelist]
    qr_captions = [code[3] for code in qr_codelist]

else:
    qr_toggle = False
    qr_codes = False
    qr_captions = False

if format_dict['text'] == 'Y':
    text_toggle = True
    texts = [text[1] for text in textlist]

    # lorem text, delete later
    lorem = '''

    Lorem ipsum dolor sit amet. Sed iusto reprehenderit ut quis voluptas ea Quis assumenda ut eius magni eum voluptate modi? Id unde libero ad pariatur sunt eos nisi possimus est omnis nisi ut internos laboriosam. Sit nesciunt ducimus et totam maxime est voluptas vero sed itaque nihil. Et expedita culpa et fuga sunt quo esse ipsam eos doloribus autem rem perferendis modi et molestiae vitae.

    Sed cumque odio eum temporibus deleniti ad consequatur consequatur aut molestias maiores. Est doloribus tenetur est dolorem ducimus sit quasi provident. Est facere expedita non expedita molestiae non error magnam vel obcaecati debitis et blanditiis reiciendis est cupiditate voluptatem ea soluta consequatur. Qui tempore ipsam eos asperiores aliquid ex unde velit est repellendus temporibus hic quidem assumenda!

    Non autem maiores aut reprehenderit nulla sit repellendus dolore ut illum incidunt ut dignissimos eaque! Ex aperiam minima eos quis dolor id consequatur eligendi ut culpa galisum. Non fugiat corporis vel doloremque dignissimos sit vero quasi sed voluptatem explicabo.

    '''
else:
    text_toggle = False
    texts = False

More dictionary creation here: this time, for team names and team codes.

RFE get rid of team codes if the user is okay with foregoing the complete team schedule in favor of individual team schedules.

In [ ]:
# key is team name
# value is team code
# also created a dictionary where the key/value pairs are swapped
team_code_dict = {}
code_team_dict = {}
for row in team_codes:
    team_name = row[0]
    team_code = row[1]
    team_code_dict[team_name] = team_code
    code_team_dict[team_code] = team_name

# key is prelim group and a number (i.e. Accra1)
# value is corresponding room (i.e. Grand Ballroom A is Accra1)
# also created dictionary where key is a playoff bracket instead
prelim_room_dict = {}
playoff_room_dict = {}
super_room_dict = {}
for i in room_assignments:
    key = i[1] + str(i[2])
    value = i[0]
    if tournament_phase == 'prelims':
        prelim_room_dict[key] = value
    elif tournament_phase == 'playoffs':
        playoff_room_dict[key] = value
    elif tournament_phase == 'super':
        super_room_dict[key] = value

Last is the error check section, to see if any of the data submitted by the user was improperly formatted.

In [ ]:
# %% error catching

# if prelim_team_count <= 3 or playoff_team_count <= 3:
#     print('Unable to produce round robins smaller than 4.')
#     sys.exit()

# if prelim_team_count >= 15 or playoff_team_count >= 15:
#     print('Unable to produce round robins larger than 14.')
#     sys.exit()

# TODO modify max_duplicates to be flexible on schedule and not hard coded
# or perhaps get rid of max duplicates since it doesn't make a lot of sense
# error check imported lists/sublists
error_check([sublist[0] for sublist in list_of_teams],
            'the team names in list_of_teams',
            max_length=26)

# error_check([sublist[2] for sublist in list_of_teams],
#             'the prelim groups in list_of_teams',
#             max_length=15,
#             )

error_check([sublist[1] for sublist in team_codes],
            'the team codes in team_codes',
            max_length=4)

try:
    error_check(prelim_group_names,
                'the groups in prelim_group_names',
                max_length=15)
except:  # TODO bare except
    None

try:
    error_check(playoff_bracket_names,
                'the brackets in playoff_bracket_names',
                max_length=15)
except:  # TODO bare except
    None

error_check([sublist[0] for sublist in room_assignments],
            'the list of rooms in room_assignments',
            max_length=15)

if error == 0:
    print('\n*** No errors detected upon import. ***')
else:
    print('\n*** Errors detected during import! ***')
    sys.exit()



No duplicates detected for the team names in list_of_teams.
No character lengths exceeded for the team names in list_of_teams.


No duplicates detected for the team codes in team_codes.
No character lengths exceeded for the team codes in team_codes.


No duplicates detected for the groups in prelim_group_names.
No character lengths exceeded for the groups in prelim_group_names.


No duplicates detected for the list of rooms in room_assignments.
No character lengths exceeded for the list of rooms in room_assignments.

*** No errors detected upon import. ***


# Function Definitions

Function for removing duplicates from a list.

In [ ]:
def remove_duplicates(alist):
    newlist = []
    [newlist.append(x) for x in alist if x not in newlist]
    return newlist

original function_definitions.py

In [ ]:
# import time
# import sys
# from pathlib import Path
# from pylatex import (Document, Tabularx, Command,
#                       NewPage, HugeText, Package,
#                      VerticalSpace)
# from pylatex.utils import NoEscape
# from cornerstone_input import format_dict

# # place additional modules here

# header = """

# Depository for a bunch of class and function definitions shared across
# prelim/playoff scripts.

# TODO what if snake seed is fed a list with odd number of teams?

# TODO what if qr_count ≠ 3?

# Created on Mon Nov 1 13:53:44 2021 Eastern Time

# @author: Victor Prieto

# """


# # starts program runtime
# start_time = time.time()
# print('\n', header)
# print('start time: %s' % time.ctime())

# # %% function definitions


# def qr_code(doc, qr_codes, qr_captions):

#     doc.append(NoEscape(r'\rowcolors{3}{white}{white}'))

#     qr_count = len(qr_codes)
#     if qr_count != 3:
#         print('WARNING: number of qr_codes does not equal 3.')
#         sys.exit()
#     header_string = 'Y' * qr_count
#     width = r"\textwidth"

#     with doc.create(Tabularx(header_string,
#                              width_argument=NoEscape(width))) as table:
#         table.add_row(qr_codes, strict=False)
#         table.add_empty_row()
#         table.add_row(qr_captions, strict=False)


# def split_list(alist, wanted_parts=1):
#     length = len(alist)
#     return [alist[i*length // wanted_parts: (i+1)*length // wanted_parts]
#             for i in range(wanted_parts)]


# def snake_seed(list_of_teams, bracket_count):
#     """
#     Parameters
#     ----------
#     list_of_teams : list
#         A list of teams for reordering into playoff brackets by snake seeding.
#     bracket_count : int
#         Number of playoff brackets to snake seed across.

#     Returns
#     -------
#     newlist : list
#         Same list of teams accepted as input, but reordered by snake seeding.
#         The list is also split into the number of playoff brackets.
#         Example:
#             (1, 2, 3, 4... 23, 24)
#             snake seeded across four brackets becomes
#             ((1, 8, 9, 16, 17, 24), (2, 7, 10, 15, 18... 12, 13, 20, 21))

#     """

#     # generates sequence of indices for snake seeding
#     sequence_length = 2 * bracket_count
#     reps = len(list_of_teams) / sequence_length
#     sequence = []
#     for i in range(0, int(sequence_length/2)):
#         for j in range(0, int(reps)):
#             sequence.append(i+j*sequence_length)
#             sequence.append((sequence_length-1-i)+j*sequence_length)

#     # quits program if odd number of teams in list_of_teams
#     if (len(list_of_teams)) % 2 == 1:
#         print('ODD NUMBER OF TEAMS DETECTED')
#         sys.exit()

#     # use sequence of indices to generate new list of teams
#     newlist = (list_of_teams[index] for index in sequence)
#     return newlist


# def alternating_rows(doc, color):
#     return doc.append(NoEscape(r'\rowcolors{3}{white}{%s}' % color))


# def duplicate_checker(alist):
#     seen = set()
#     for x in alist:
#         if x in seen:
#             return True
#         seen.add(x)
#     return False


# def start_latex(filename_input, docname, title=True, fontsize=False):

#     Path("./output/").mkdir(parents=True, exist_ok=True)
#     file_path = r'./output/'
#     Path(file_path).mkdir(parents=True, exist_ok=True)
#     filename = file_path + filename_input        
    
#     geometry_options = {
#         "includeheadfoot": True
#     }

#     doc = Document(filename, geometry_options=geometry_options)

#     # this changed font sizes, but it doesn't work so well
#     if fontsize is True:
#         doc = Document(filename_input, geometry_options=geometry_options,
#                        font_size='')
#         doc.append(Command('fontsize', arguments=['12', '12']))
#         doc.append(Command('selectfont'))

#     doc.preamble.append(Command('title', format_dict['tournament name']))
#     doc.preamble.append(Command('author', format_dict['tournament location']))
#     doc.preamble.append(Command('date', format_dict['tournament date']))
    
#     doc.packages.append(Package('qrcode'))  # add qrcode to packages
#     doc.packages.append(Package('fancyhdr'))  # add qrcode to packages

#     doc.append(NoEscape(r'\newcolumntype{Y}{>{\centering\arraybackslash}X}'))

#     # TODO vertically center docname text on page
#     if title is True:
#         doc.append(NoEscape(r'\maketitle'))
#         doc.append(VerticalSpace('48pt'))
#         doc.append(NoEscape(r'\begin{center}'))
#         doc.append(HugeText(docname))
#         doc.append(NoEscape(r'\end{center}'))
#         doc.append(NewPage())
    
#     doc.append(NoEscape(r'\pagestyle{fancy}'))
#     doc.append(NoEscape(r'\fancyhf{}'))
#     date = '{' + format_dict['tournament date'] + '}'
#     name = '{' + format_dict['tournament name'] + '}'
#     doc.append(NoEscape(fr'\rhead{date}'))
#     doc.append(NoEscape(fr'\lhead{name}'))
#     doc.append(NoEscape(r'\rfoot{Created by Cornerstone.}'))
        
#     return doc


# def close_latex(filename_input, doc_input):
#     file_path = r'./output/'
#     filename = file_path + filename_input
#     doc_input.generate_tex()
#     doc_input.generate_pdf()

#     file = open(filename + '.tex')
#     latex_string = file.read()
#     bad_string = '\\usepackage{lastpage}%'
#     good_string = '\\usepackage[table]{xcolor}%'
#     # print(latex_string)
#     latex_string = latex_string.replace(bad_string, good_string)

#     # for visual debugging
#     # print('\n\n', latex_string[0:700], '\n\n')

#     with open(f'{filename}.tex', 'w') as f:
#         f.write(latex_string)


# # %% classes, mostly not in use
# # TODO figure out class inheritance for team/prelim_team/playoff_team
# class team:
#     def __init__(self,
#                  name,
#                  code):
#         self.name = name
#         self.code = code


# class prelim_team(team):
#     def __init__(self,
#                  name,
#                  code,
#                  prelim_group,
#                  prelim_seed):
#         super().__init__(name, code)
#         self.prelim_group = prelim_group
#         self.prelim_seed = prelim_seed


# class playoff_team():
#     def __init__(self,
#                  name,
#                  code,
#                  prelim_group,
#                  playoff_bracket,
#                  playoff_seed):  # playoff seed also the prelim finish
#         self.name = name
#         self.code = code
#         self.prelim_group = prelim_group
#         self.playoff_bracket = playoff_bracket
#         self.playoff_seed = playoff_seed

# # %% more function definitions


# def header_stringify(teamcount, tabularx=False):
#     """

#     Parameters
#     ----------
#     teamcount : int
#         Number of teams in the round robin.
#     tabularx : bool, optional
#         Changes header string to account for newcolumntype Y, used in tabularx.
#         The default is False.

#     Returns
#     -------
#     A string for pylatex to use when defining the table.
#     Example: '|l|cc|cc|cc|cc|'

#     If tabularx is set to True, returns a string that utilizes newcolumntype Y.
#     Example: '|l|Y|Y|Y|Y|Y|Y|'

#     """

#     roomcount = teamcount // 2

#     header_string = '|l' + ('|cc' * roomcount + '|')
#     if teamcount % 2 == 1:
#         header_string = header_string + 'c|'

#     if tabularx is True:
#         header_string = '|l' + ('|Y' * teamcount + '|')

#     return header_string


# # %% prints runtime
# print('end time: %s' % time.ctime())
# print("--- %s seconds ---" % '%.3f' % (time.time() - start_time))
# print("--- %s minutes ---" % '%.3f' % (time.time()/60 - start_time/60))

# Standard Scheduler Function

Created on Mon Nov 22 2021

Definition of functions used for creating a table for a round robin schedule,
between 4 and 14 teams.

TODO if possible, fix pylatex table lines and shading in this function

First: retrieve the round robin templates stored on the cornerstone github page. The templates will automatically populate a dictionary.

Each key is the name of the sheet corresponding to the round robin template: rr_6 is a basic 6 team round robin, rr_12 is for 12 teams, rr_crossover_A8 is the 8 team playoff roundrobin including crossovers (there are two possible crossovers, A and B, based on [Dwight Wynne's spreadsheet of schedule formats](https://https://docs.google.com/spreadsheets/d/1BPF6jFvhkh5qe7eesGFGU9KH6FVps2tS3YeseFocpIE/edit#gid=16)).

The dictionary values are the pandas dataframes for each template.

These templates will be used in the standard scheduler function and retrieved based on the key name.

In [ ]:
url = "https://github.com/victorprieto2265/cornerstone/blob/main/templates/rr_schedules/rr_templates.xlsx?raw=true"
rr_template_dict = pd.read_excel(url,
                                 sheet_name = None)
rr_template_dict.keys()

dict_keys(['rr_3', 'rr_4', 'rr_5', 'rr_6', 'rr_7', 'rr_8', 'rr_9', 'rr_10', 'rr_11', 'rr_12', 'rr_13', 'rr_14', 'rr_crossover_B3', 'rr_crossover_A6', 'rr_crossover_B6', 'rr_crossover_B7', 'rr_crossover_A8', 'rr_crossover_B8'])

DONE 

*  testing of proper schedule retrieval from rr_template_dict

TODO

*  fix the crossover templates in the excel file, which right now are a mess 

In [ ]:
teamcount = 8
crossover = 'A'

if crossover is False:
  key = f'rr_{teamcount}'
else:
  key = f'rr_crossover_{crossover}{teamcount}'

rr_schedule = rr_template_dict[key].values.tolist()

print(*rr_schedule, sep='\n')

['A1', 'B2', 'C1', 'D2', 'B1', 'A2', 'D1', 'C2']
['C1', 'A2', 'A1', 'C2', 'D1', 'B2', 'B1', 'D2']
['B1', 'C2', 'D1', 'A2', 'A1', 'D2', 'C1', 'B2']
['A1', 'D1', 'B2', 'C2', 'B1', 'C1', 'A2', 'D2']
['B2', 'D2', 'B1', 'D1', 'A2', 'C2', 'A1', 'C1']
['C1', 'D1', 'A1', 'B1', 'C2', 'D2', 'A2', 'B2']


Definition of the standard scheduler function is below.

In [ ]:
def standard_schedule(group_name, teamcode_dict, room_dict, roundstart=1,
                      crossover=False, superplayoff=False):
    """


    Parameters
    ----------
    groupname : string
        String of the prelim group or playoff bracket name.
        Example: Lilongwe
    teamcode_dict : dict
        k : v = a group/bracket code : teamcode corresponding to that code
        example: Lilongwe4 = MBA
    room_dict : dict
        k : v = a group/bracket code : room corresponding to that code
        example: Lilongwe2 = Suite 1132
    roundstart : int, optional
        Number for the first round in the schedule. The default is 1.

    Returns
    -------
    schedule_grid : list
        Latex-friendly table. First row = table header, remaining rows =
        schedule for each round.

    """

    # produce teamlist
    teamlist = [k for k, v in list(teamcode_dict.items())
                if k.startswith(group_name)]
    sorted_list = sorted(teamlist)
    teamlist = [teamcode_dict[i] for i in sorted_list]

    # identify number of teams in teamlist and appropriate number of rooms
    teamcount = len(teamlist)
    roomcount = teamcount // 2

    # retrieve correct rr_template from rr_template_dict
    if crossover is False:
      key = f'rr_{teamcount}'
    else:
      key = f'rr_crossover_{crossover}{teamcount}'

    rr_schedule = rr_template_dict[key].values.tolist()

    # produce roomlist
    roomlist = [k for k, v in list(room_dict.items())
                if k.startswith(group_name)]
    roomlist = [room_dict[i] for i in sorted(roomlist)]

    # populate header with room names
    rooms = []
    header = ['']
    for index in range(roomcount):
        room = roomlist[index]
        header.append(room)

    # the last element in the header should be BYE if teamcount is odd
    if teamcount % 2 == 1:
        header.append('BYE')

    # use rr_schedule template and roundstart to produce rest of schedule grid
    schedule_grid = [header]
    for round_num, row in enumerate(rr_schedule):
        round_num += roundstart
        new_row = [f'Round {round_num}']
        for index in row:
            code = group_name + str(index)
            team = teamcode_dict[code]
            new_row.append(team)
        schedule_grid.append(new_row)

    return schedule_grid

## Individual Schedule Functions 

# Prelim Scheduler

This section reads the list of prelim teams and information from the tournament format dictionary to produce a set of prelim schedules.

dictionary creation

prelim group names

In [ ]:
prelim_group_names = [sublist[1] for sublist in list_of_teams]
prelim_group_names = remove_duplicates(prelim_group_names)

# key is a prelim group name and a seed (i.e. Belmopan6)
# value is the corresponding team (i.e. Great Valley A for Belmopan6)
# also created a dictionary where values are team codes (i.e. GVA)
prelim_team_dict = {}
prelim_teamcode_dict = {}
team_group_dict = {}
teamcode_group_dict = {}
for i in list_of_teams:
    # key = i[2] + str(i[3])
    key = i[1] + str(i[2])
    value = i[0]  # team name
    prelim_team_dict[key] = value
    # also created dictionary where k/v pairs are swapped
    team_group_dict[value] = key
    # value = i[1]  # team code
    value = team_code_dict[value]
    prelim_teamcode_dict[key] = value
    # also created dictionary where k/v pairs are swapped
    teamcode_group_dict[value] = key

# key is prelim group and a number (i.e. Accra1)
# value is corresponding room (i.e. Grand Ballroom A is Accra1)
# also created dictionary where key is a playoff bracket instead
prelim_room_dict = {}
for i in room_assignments:
    key = i[1] + str(i[2])
    value = i[0]
    prelim_room_dict[key] = value

full schedule grid creation

In [ ]:
full_schedule_grid = []

# perform this process for each prelim group, add to full_schedule_grid
for group_name in prelim_group_names:
    schedule_grid = (standard_schedule(group_name,
                                       prelim_teamcode_dict,
                                       prelim_room_dict))
    full_schedule_grid.append(schedule_grid)

To make sure the full_schedule_grid was populated correctly, we'll print it out line by line.

In [ ]:
# for visual debugging
for index, prelim_group in enumerate(prelim_group_names):
    print(f'\nPrelim Group: {prelim_group}')
    print(*full_schedule_grid[index], sep='\n')


Prelim Group: Spooky
['', 'Room 205', 'Room 206', 'Room 207', 'BYE']
['Round 1', 'HNB', 'NBB', 'VSS', 'DRA', 'KPB', 'HTC', 'CHE']
['Round 2', 'NBB', 'DRA', 'HNB', 'HTC', 'VSS', 'CHE', 'KPB']
['Round 3', 'DRA', 'HTC', 'NBB', 'CHE', 'HNB', 'KPB', 'VSS']
['Round 4', 'CHE', 'HTC', 'KPB', 'DRA', 'VSS', 'NBB', 'HNB']
['Round 5', 'KPB', 'CHE', 'VSS', 'HTC', 'HNB', 'DRA', 'NBB']
['Round 6', 'VSS', 'KPB', 'HNB', 'CHE', 'NBB', 'HTC', 'DRA']
['Round 7', 'HNB', 'VSS', 'NBB', 'KPB', 'DRA', 'CHE', 'HTC']

Prelim Group: Scary
['', 'Room 232', 'Room 238', 'Room 239', 'BYE']
['Round 1', 'KPC', 'NBA', 'ILT', 'HTB', 'DRB', 'MPB', 'HNA']
['Round 2', 'NBA', 'HTB', 'KPC', 'MPB', 'ILT', 'HNA', 'DRB']
['Round 3', 'HTB', 'MPB', 'NBA', 'HNA', 'KPC', 'DRB', 'ILT']
['Round 4', 'HNA', 'MPB', 'DRB', 'HTB', 'ILT', 'NBA', 'KPC']
['Round 5', 'DRB', 'HNA', 'ILT', 'MPB', 'KPC', 'HTB', 'NBA']
['Round 6', 'ILT', 'DRB', 'KPC', 'HNA', 'NBA', 'MPB', 'HTB']
['Round 7', 'KPC', 'ILT', 'NBA', 'DRB', 'HTB', 'HNA', 'MPB']

Prelim

# Class Definitions for fpdf2

There are five types of tables that Cornerstone will need to generate:
*  the full schedule
*  the individualized team schedule
*  the individualized room schedule
*  the team index
*  the playoff quickstart schedule (first playoff room for all teams)

The last one will not be needed for the prelim writer, but will be necessary for playoffs (and superplayoffs if needed).

For all of these tables, we will create a unique class via fpdf2 for writing the table onto a PDF. 

In [ ]:
class PDF(FPDF):

    def full_schedule(self, header, rows,
                      header_color=(28, 97, 57), row_color=(255, 248, 206)):
      
      header_count = len(header)
      if header[-1] == 'BYE':
        cell_count = header_count * 2 - 2
      else:
        cell_count = header_count * 2 - 1
      col_width = pdf.epw / cell_count  # distribute columns evenly

      header_widths = [col_width * 2] * (header_count - 2)  # populate list with col widths
      header_widths.insert(0, col_width)
      header_widths.append(col_width)

      cell_widths = [col_width] * cell_count

      self.set_fill_color(*header_color)  # * required to unpack tuple
      self.set_text_color(255)
      self.set_draw_color(36, 36, 36)
      self.set_line_width(0.3)
      self.set_font(style="B")
      for col_width, heading in zip(header_widths, header):
          self.cell(col_width, 7, heading, border=1, align="C", fill=True)
      self.ln()

      # Color and font restoration:
      self.set_fill_color(*row_color)  # * required to unpack tuple
      self.set_text_color(0)
      self.set_font()
      fill = False
      for row in rows:
        for index, item in enumerate(row):          
          if index == 0:
            self.set_font(style="B")
          else:
            self.set_font()
          self.cell(cell_widths[index],
                    7,
                    item,
                    border="LR",
                    align="C",
                    fill=fill)
        self.ln()
        fill = not fill
        # self.cell(sum(col_widths), 0, "", "T")  # this was in the original but broke things so it's commented out for now

      self.set_fill_color(*header_color)  # * required to unpack tuple
      self.set_text_color(255)
      self.set_draw_color(36, 36, 36)
      self.set_line_width(0.3)
      self.set_font(style="B")
      for col_width, heading in zip(header_widths, header):
          self.cell(col_width, 7, heading, border=1, align="C", fill=True)

      self.ln(14)

In [ ]:
class PDF(PDF):

    def fancy_table(self, header, rows, col_widths=(42, 39, 35, 42)):
        # Colors, line width and bold font:
        self.set_fill_color(255, 100, 0)
        self.set_text_color(255)
        self.set_draw_color(255, 0, 0)
        self.set_line_width(0.3)
        self.set_font(style="B")
        for col_width, heading in zip(col_widths, header):
            self.cell(col_width, 7, heading, border=1, align="C", fill=True)
        self.ln()
        # Color and font restoration:
        self.set_fill_color(224, 235, 255)
        self.set_text_color(0)
        self.set_font()
        fill = False
        for row in rows:
            self.cell(col_widths[0], 6, row[0], border="LR", align="L", fill=fill)
            self.cell(col_widths[1], 6, row[1], border="LR", align="L", fill=fill)
            self.cell(col_widths[2], 6, row[2], border="LR", align="R", fill=fill)
            self.cell(col_widths[3], 6, row[3], border="LR", align="R", fill=fill)
            self.ln()
            fill = not fill
        self.cell(sum(col_widths), 0, "", "T")

We will also need to generate class definitions for the page header and page footer.

In [315]:
class PDF(PDF):

    def page_header(self, tournament_name, tournament_date):
                
        # Setting font: bold times 11
        font_size = 11
        self.set_font("times", style="I", size=font_size)

        # Print tournament name in upper left:
        self.cell(None, 2, tournament_name, new_x="LEFT")

        # Create empty cell with bottom border (can modify for centered text)
        self.cell(0, None, " ", "B", align="C")

        # Print tournament date in upper right
        self.cell(0, 2, tournament_date, align="R")

        # Performing a line break:
        self.ln(20)

    def page_footer(self):
        
        # # putting in a logo:
        # self.image("../docs/fpdf2-logo.png", 10, 8, 33)
        
        # Position cursor at left and 1.5 cm from bottom:
        self.set_x(0)
        self.set_y(-35)
        
        # Setting font: times italic 11
        font_size = 11
        self.set_text_color(0)
        self.set_font("times", style="I", size=font_size)
        
        # Print footer text:
        self.cell(0, 6, "Created by Cornerstone", border="T", align="R")


# Prelim PDF Writer

In [316]:
print(format_dict['tournament name'])
print(format_dict['tournament location'])
print(format_dict['tournament date'])

pdf = PDF(format="letter")  # default is A4, need to specify US letter

# title page
pdf.add_page()
pdf.page_header(format_dict['tournament name'],
                format_dict['tournament date'])

header = full_schedule_grid[0][0]
grid = full_schedule_grid[0][1:]

for index, grid in enumerate(full_schedule_grid):
  
  header = grid[0]
  schedule = grid[1:]
  pdf.full_schedule(header, schedule)

pdf.page_footer()

pdf.output("test_output.pdf")
pdf_bytes = pdf.output()
base64_pdf = b64encode(pdf_bytes).decode("utf-8")

LIFT XXI
Uniondale, NY
October 29, 2022


In [317]:
# Diplay a download button:
display(HTML(f'<a style="font-size: 30px" download="test_output.pdf" href="data:application/pdf;base64,{base64_pdf}">Click to download PDF</a>'))

In [318]:
stop here

SyntaxError: ignored

 Prelim Analysis

# Playoff Scheduler

# Playoff PDF Writer

# Superplayoff Scheduler

# Superplayoff PDF Writer 

# NSC Scheduler

# *** Workspace below this point ***



---



# Importing csv files from Github
This will be useful later when grabbing templates for the scheduling functions.

For more info on loading external files into Colabs, see [this article](https://towardsdatascience.com/7-ways-to-load-external-data-into-google-colab-7ba73e7d5fc7).

In [ ]:
filename = 'templates/rr_schedules/rr_10.xlsx'

df = pd.read_csv('https://raw.githubusercontent.com/victorprieto2265/cornerstone/main/archive/brackets.csv')

# get_all_values gives a list of rows.
print(df)

What about uploading excel files? Will that work?

In [ ]:
url = "https://github.com/victorprieto2265/cornerstone/blob/main/templates/rr_schedules/rr_templates.xlsx?raw=true"
db=pd.read_excel(url, sheet_name=None)

print(type(db))
print(db.keys())

# Compiling latex files (incomplete)

Load a sample Latex file from my github repo. In this case, it's team index, which does not have any QR codes.

In [ ]:
import base64
import requests

url = 'https://raw.githubusercontent.com/victorprieto2265/cornerstone/main/output/prelim_team_index.tex'

req = requests.get(url)
req = req.text

In [ ]:
from pdflatex import PDFLaTeX
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
filename = 'prelim_team_index.tex'
with open(f'/content/drive/My Drive/{filename}', 'w') as f:
  f.write(req)
!cat /content/drive/My\ Drive/prelim_team_index.tex

In [ ]:
!sudo apt install cm-super dvipng texlive-latex-extra texlive-latex-recommended

In [ ]:
!pip install pdflatex
from pdflatex import PDFLaTeX
pdfl = PDFLaTeX.from_texfile(req)
pdf, log, completed_process = pdfl.create_pdf(keep_pdf_file=True, keep_log_file=False)


In [ ]:
# drive.flush_and_unmount()